# queue

> Selection queue route handlers for Phase 1

In [ ]:
#| default_exp routes.queue

In [ ]:
#| export
from typing import Tuple, Dict, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.routes.core import (
    WorkflowStateStore, _get_step_state, _update_step_state, _build_queue_response
)
from cjm_transcript_source_select.components.preview_panel import (
    _render_preview_panel
)
from cjm_transcript_source_select.services.source import SourceService
from cjm_transcript_source_select.services.source_utils import (
    select_all_in_group, reorder_sources
)

## Add/Remove Handlers

In [ ]:
#| export
def _handle_selection_add(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    record_id: str,  # Job ID to add
    provider_id: str,  # Plugin name for the source
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component with OOB stats, optionally with OOB source list
    """Add a source to the selection queue."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    # Check if already selected
    if not any(s.get("record_id") == record_id for s in selected_sources):
        selected_sources.append({"record_id": record_id, "provider_id": provider_id})
        _update_step_state(state_store, workflow_id, session_id, selected_sources)
    
    return _build_queue_response(state_store, workflow_id, source_service, session_id, selected_sources, urls)

In [ ]:
#| export
def _handle_selection_remove(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    record_id: str,  # Job ID to remove
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component with OOB stats, optionally with OOB source list
    """Remove a source from the selection queue."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    # Remove the item
    selected_sources = [s for s in selected_sources if s.get("record_id") != record_id]
    _update_step_state(state_store, workflow_id, session_id, selected_sources)
    
    return _build_queue_response(state_store, workflow_id, source_service, session_id, selected_sources, urls)

## Reorder/Clear Handlers

In [ ]:
#| export
async def _handle_selection_reorder(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SelectionUrls,  # URL bundle for rendering
):  # Updated queue component
    """Reorder items in the selection queue based on SortableJS result."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    # Extract new order from form data (hidden inputs sent in DOM order after drag-drop)
    form_data = await request.form()
    new_order_ids = form_data.getlist("item")
    
    selected_sources = reorder_sources(selected_sources, new_order_ids)
    _update_step_state(state_store, workflow_id, session_id, selected_sources)
    
    return _build_queue_response(
        state_store, workflow_id, source_service, session_id, selected_sources, urls,
        include_stats=False, include_source_list=False,
    )

In [ ]:
#| export
def _handle_selection_clear(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component with OOB stats, optionally with OOB source list
    """Clear all items from the selection queue."""
    session_id = get_session_id(sess)
    _update_step_state(state_store, workflow_id, session_id, [])
    
    return _build_queue_response(state_store, workflow_id, source_service, session_id, [], urls)

## Select All Handler

In [ ]:
#| export
def _handle_selection_select_all(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    group_key: str,  # Group key to select all transcriptions for
    grouping_mode: str,  # Current grouping mode: "media_path" or "batch_id"
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component with OOB stats, optionally with OOB source list
    """Select all transcriptions for a given group."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    all_transcriptions = source_service.query_transcriptions(limit=500)
    selected_sources = select_all_in_group(all_transcriptions, group_key, grouping_mode, selected_sources)
    
    _update_step_state(state_store, workflow_id, session_id, selected_sources=selected_sources)
    
    return _build_queue_response(
        state_store, workflow_id, source_service, session_id, selected_sources, urls, grouping_mode=grouping_mode,
    )

## Preview Handler

In [ ]:
#| export
def _handle_selection_preview(
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    record_id: str,  # Job ID to preview
    provider_id: str,  # Plugin name for the source
):  # Full preview panel component (collapsible, open with content)
    """Get preview panel for a selected source."""
    # Get the transcription from the source service
    source_block = source_service.get_transcription_by_id(record_id, provider_id)
    
    if not source_block:
        # Return preview panel with error state
        return _render_preview_panel(
            preview_record_id=record_id,
            preview_text="Could not load preview content.",
            is_open=True
        )
    
    # Return the full preview panel with content and open state
    return _render_preview_panel(
        preview_record_id=record_id,
        preview_text=source_block.text,
        is_open=True
    )

## Router

In [ ]:
#| export
def init_queue_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    prefix: str,  # Route prefix (e.g., "/workflow/selection/queue")
    urls: SelectionUrls,  # URL bundle for rendering (populated after all routers created)
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize queue management routes."""
    router = APIRouter(prefix=prefix)

    @router
    def add(request, sess, record_id: str, provider_id: str):
        """Add a source to the selection queue."""
        return _handle_selection_add(
            state_store, workflow_id, source_service,
            request, sess, record_id, provider_id, urls=urls,
        )

    @router
    def remove(request, sess, record_id: str):
        """Remove a source from the selection queue."""
        return _handle_selection_remove(
            state_store, workflow_id, source_service,
            request, sess, record_id, urls=urls,
        )

    @router
    async def reorder(request, sess):
        """Reorder items in the selection queue."""
        return await _handle_selection_reorder(
            state_store, workflow_id, source_service,
            request, sess, urls=urls,
        )

    @router
    def clear(request, sess):
        """Clear all items from the selection queue."""
        return _handle_selection_clear(
            state_store, workflow_id, source_service,
            request, sess, urls=urls,
        )

    @router
    def select_all(request, sess, group_key: str, grouping_mode: str = "media_path"):
        """Select all transcriptions for a given group."""
        return _handle_selection_select_all(
            state_store, workflow_id, source_service,
            request, sess, group_key, grouping_mode, urls=urls,
        )

    @router
    def preview(request, record_id: str, provider_id: str):
        """Get preview content for a selected source."""
        return _handle_selection_preview(source_service, request, record_id, provider_id)

    routes = {
        "add": add,
        "remove": remove,
        "reorder": reorder,
        "clear": clear,
        "select_all": select_all,
        "preview": preview,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()